In [1]:
from powersimdata.scenario.scenario import Scenario
from powersimdata.scenario.scenario_info import ScenarioInfo
import json
import pandas as pd

In [2]:
from powersimdata.scaling.clean_capacity_scaling.auto_capacity_scaling import CollaborativeStrategyManager,\
IndependentStrategyManager, AbstractStrategyManager, TargetManager, ResourceManager, Resource

In [3]:
scenario_string = '394'
targets_info_location ='Eastern Scenario Take 2.csv'

Load a scenario to that we will extract resource information from

In [4]:
scenario = Scenario(scenario_string)

SCENARIO: base | EasternBase_2016_Final_2020Feb_take3

--> State
analyze


Inherit from ScenarioInfo to add a couple helper methods - likely will change this later as ScenarioInfo evolves

In [5]:
class ScenarioResources(ScenarioInfo):
    def get_available_resources(self, area):
        loadzone_set = self.area_to_loadzone(area)
        available_resources = self.grid.plant[self.grid.plant['zone_name'].isin(loadzone_set)]['type'].unique()
        return available_resources.tolist()

    def get_profiled_resources(self):
        return self.profile.keys()

Create a scenario_info object from which we calculate resource properties 

In [6]:
scenario_info = ScenarioResources(scenario)

--> Loading PG
--> Loading grid
Loading bus
Loading plant
Loading heat_rate_curve
Loading gencost_before
Loading gencost_after
Loading branch
Loading dcline
Loading sub
Loading bus2sub
--> Loading demand
--> Loading solar
--> Loading wind
--> Loading hydro


Read in external parameters for region capacity targets

In [7]:
eastern = pd.read_csv(targets_info_location)
eastern

,region_name,2016_demand,2030_2016_demand_scalar,total_demand,ce_target_fraction,ce_category,allowed_resources,external_ce_historical_amount,solar_percentage
0,Alabama,83239040,1.028367,8.560028e+07,0.00,NaN,"solar,wind",NaN,NaN
1,Arkansas,49463250,1.165510,5.764994e+07,0.00,NaN,"solar,wind",NaN,NaN
2,Connecticut,32931880,0.945433,3.113488e+07,0.44,Renewables,"geothermal, hydro, solar, wind",NaN,NaN
3,Delaware,12381650,1.165510,1.443094e+07,0.25,Renewables,"geothermal, hydro, solar, wind",NaN,NaN
4,Florida,253533010,1.149474,2.914297e+08,0.00,NaN,"solar,wind",NaN,NaN
5,Georgia,171946010,1.028367,1.768236e+08,0.00,NaN,"solar,wind",NaN,NaN
6,Iowa,39669870,1.216553,4.826049e+07,0.00,NaN,"solar,wind",NaN,NaN
7,Illinois,165066780,1.069337,1.765121e+08,0.25,Renewables,"hydro, solar, wind",NaN,NaN
8,Indiana,96674130,1.173606,1.134574e+08,0.10,Clean,"geothermal, hydro, nuclear, solar, wind",NaN,NaN
9,Kansas,51402860,1.087356,5.589320e+07,0.20,Renewables,"hydro, solar, wind",NaN,NaN


Clean up the missing data with appropriate defaults

In [8]:
eastern['external_ce_historical_amount'] = eastern['external_ce_historical_amount'].fillna(0)
eastern['allowed_resources'] = eastern['allowed_resources'].fillna('solar,wind')
eastern['ce_category'] = eastern['ce_category'].fillna('TBD')
eastern['solar_percentage'] = eastern['solar_percentage'].fillna('None')
eastern

,region_name,2016_demand,2030_2016_demand_scalar,total_demand,ce_target_fraction,ce_category,allowed_resources,external_ce_historical_amount,solar_percentage
0,Alabama,83239040,1.028367,8.560028e+07,0.00,TBD,"solar,wind",0.0,None
1,Arkansas,49463250,1.165510,5.764994e+07,0.00,TBD,"solar,wind",0.0,None
2,Connecticut,32931880,0.945433,3.113488e+07,0.44,Renewables,"geothermal, hydro, solar, wind",0.0,None
3,Delaware,12381650,1.165510,1.443094e+07,0.25,Renewables,"geothermal, hydro, solar, wind",0.0,None
4,Florida,253533010,1.149474,2.914297e+08,0.00,TBD,"solar,wind",0.0,None
5,Georgia,171946010,1.028367,1.768236e+08,0.00,TBD,"solar,wind",0.0,None
6,Iowa,39669870,1.216553,4.826049e+07,0.00,TBD,"solar,wind",0.0,None
7,Illinois,165066780,1.069337,1.765121e+08,0.25,Renewables,"hydro, solar, wind",0.0,None
8,Indiana,96674130,1.173606,1.134574e+08,0.10,Clean,"geothermal, hydro, nuclear, solar, wind",0.0,None
9,Kansas,51402860,1.087356,5.589320e+07,0.20,Renewables,"hydro, solar, wind",0.0,None


Create Independent and Collaborative Strategy objects and set the simulation hours to be a full year

In [9]:
independent_strategy_manager = IndependentStrategyManager()
collaborative_strategy_manager = CollaborativeStrategyManager()
AbstractStrategyManager.set_next_sim_hours(8784)

Populate strategy managers with targets created from external data

In [10]:
independent_strategy_manager.targets_from_data_frame(eastern)
collaborative_strategy_manager.targets_from_data_frame(eastern)

For each target region, add the available resources and also create allowed resources that are not currently present

In [11]:
start_time = '2016-01-01 00:00:00'
end_time = '2016-12-31 23:00:00'

for region_name in independent_strategy_manager.targets:
    print()
    print(region_name)
    print()
    allowed_resources = set(independent_strategy_manager.targets[region_name].allowed_resources)
    available_resources = set(scenario_info.get_available_resources(region_name))
    all_resources = available_resources.union(allowed_resources)
    
    resources = ResourceManager()
    resources.pull_region_resource_info(region_name, scenario_info, int(scenario.info['id']),
                      all_resources, start_time, end_time)
    
    independent_strategy_manager.targets[region_name].add_resource_manager(resources)
    collaborative_strategy_manager.targets[region_name].add_resource_manager(resources)


Alabama

Added resource wind!

Added resource nuclear!

Added resource coal!

Added resource solar!

Added resource hydro!

Added resource other!

Added resource dfo!

Added resource ng!


Arkansas

Added resource wind!

Added resource nuclear!

Added resource coal!

Added resource solar!

Added resource hydro!

Added resource other!

Added resource dfo!

Added resource ng!


Connecticut

Added resource wind!

Added resource nuclear!

Added resource coal!

Added resource solar!

Added resource hydro!

Added resource other!



C:\Users\dmuldrew\AppData\Local\Continuum\anaconda3\lib\site-packages\powersimdata\scenario\scenario_info.py:135: UserWarning: No such type of generator in the area specified!
  warnings.warn('No such type of generator in the area specified!')


No existing resource geothermal!


C:\Users\dmuldrew\AppData\Local\Continuum\anaconda3\lib\site-packages\powersimdata\scenario\scenario_info.py:135: UserWarning: No such type of generator in the area specified!
  warnings.warn('No such type of generator in the area specified!')


Added resource geothermal!

Added resource dfo!

Added resource ng!


Delaware

Added resource wind!

Added resource coal!

Added resource solar!

Added resource other!



...

C:\Users\dmuldrew\AppData\Local\Continuum\anaconda3\lib\site-packages\powersimdata\scenario\scenario_info.py:135: UserWarning: No such type of generator in the area specified!
  warnings.warn('No such type of generator in the area specified!')


No existing resource geothermal!


C:\Users\dmuldrew\AppData\Local\Continuum\anaconda3\lib\site-packages\powersimdata\scenario\scenario_info.py:135: UserWarning: No such type of generator in the area specified!
  warnings.warn('No such type of generator in the area specified!')


Added resource geothermal!

Added resource dfo!

Added resource ng!


West Virginia

Added resource wind!

Added resource coal!

Added resource solar!

Added resource other!

Added resource hydro!

Added resource dfo!

Added resource ng!



Socket exception: An existing connection was forcibly closed by the remote host (10054)


Calculate the Independent Strategy next capacities along with intermediate values

In [ ]:
independent_next_capacities = independent_strategy_manager.data_frame_of_next_capacities()
independent_next_capacities

In [ ]:
#independent_next_capacities.to_excel("Eastern_Independent.xlsx")

Calculate the Collaborative Strategy next capacities along with intermediate values

In [ ]:
collaborative_next_capacities = collaborative_strategy_manager.data_frame_of_next_capacities()
collaborative_next_capacities

Export all target properties as rows: first, use jsonpickle to convert targets to json,
and second, use json_normalize to flatten this json hierarchy into a dataframe

In [ ]:
import jsonpickle
from pandas.io.json import json_normalize

cap_planning_df = pd.DataFrame()
for tar in independent_strategy_manager.targets:
    target_df = json_normalize(json.loads(jsonpickle.encode(independent_strategy_manager.targets[tar], unpicklable=False)))
    cap_planning_df = cap_planning_df.append(target_df, sort=False)
cap_planning_df = cap_planning_df.set_index('region_name')
cap_planning_df

In [ ]:
#cap_planning_df.to_excel("Eastern_Capacity_Inputs.xlsx")

In [ ]:
unchanged_capacities = cap_planning_df[[
    "resources.resources.coal.prev_capacity",
    "resources.resources.dfo.prev_capacity",
    "resources.resources.geothermal.prev_capacity",
    "resources.resources.hydro.prev_capacity",
    "resources.resources.ng.prev_capacity",
    "resources.resources.nuclear.prev_capacity", 
    "resources.resources.other.prev_capacity"
]].rename(columns={
    "resources.resources.coal.prev_capacity":"Coal",
    "resources.resources.dfo.prev_capacity":"DFO",
    "resources.resources.geothermal.prev_capacity":"Geo-thermal",
    "resources.resources.hydro.prev_capacity":"Hydro",
    "resources.resources.ng.prev_capacity":"Natural Gas",
    "resources.resources.nuclear.prev_capacity":"Nuclear", 
    "resources.resources.other.prev_capacity":"Other"})
unchanged_capacities

In [ ]:
next_capacities = independent_next_capacities[["next_solar_capacity", "next_wind_capacity"]]
next_capacities

In [ ]:
merged_capacities = pd.concat([unchanged_capacities,next_capacities], axis=1)
merged_capacities = merged_capacities.fillna(0.0)
merged_capacities

In [ ]:
updated_capacities.to_excel("Next_Capacity_Outputs.xlsx")